# Additional Usage with psycopg2

### Imports
```
import psycopg2
import age
```
### Connect to PostgreSQL(with AGE extention)

In [1]:
GRAPH_NAME = "test_graph4"

In [2]:
import psycopg2
import age

# Connect to PostgresDB with AGE
conn = psycopg2.connect(host="172.17.0.3", port="5432", dbname="postgres", user="postgres", password="agens")
cursor = conn.cursor()

# load AGE extention
cursor.execute("LOAD 'age';")
cursor.execute("SET search_path = ag_catalog, '$user', public;")

# Check if age_graph named GRAPH_NAME exists
cursor.execute("SELECT count(*) FROM ag_graph WHERE name=%s", (GRAPH_NAME,))

if cursor.fetchone()[0] == 0 :
# if age_graph named GRAPH_NAME, create that
    cursor.execute("SELECT create_graph(%s);", (GRAPH_NAME,))


### Create & Query Vertices

In [3]:
# Create Vertices
age.execCypher(cursor, GRAPH_NAME, "CREATE (n:Person {name: 'Joe'})")
age.execCypher(cursor, GRAPH_NAME, "CREATE (n:Person {name: 'Smith'})")
age.execCypher(cursor, GRAPH_NAME, "CREATE (n:Person {name: %s})", ('Jack',))
age.execCypher(cursor, GRAPH_NAME, "CREATE (n:Person {name: 'Andy', title: 'Developer'})")
age.execCypher(cursor, GRAPH_NAME, "CREATE (n:Person {name: %s, title: %s})", ('Tom','Developer',))
conn.commit()

# Query Vertices
age.queryCypher(cursor, GRAPH_NAME, "MATCH (n:Person) RETURN n")

# parse each row in cursor
print("---[Parse & print vertices row by row]---")
for row in age.getRows(cursor):
    print(row.id, row.label, row.properties)

    
# Query Vertices with parameters
age.queryCypher(cursor, GRAPH_NAME, "MATCH (n:Person {name:%s}) RETURN n", ("Andy",))

# Parse full data in cursor
print("---[Parse all vertices in result set]---")
graph = age.buildGraph(cursor)

print("Count of queried vertices:", graph.size())

print("---[Print vertices in Graph object]---")
for row in graph:
    print(row.id, row.label, row.properties)
    

---[Parse & print vertices row by row]---
844424930131969 Person {'name': 'Joe'}
844424930131970 Person {'name': 'Smith'}
844424930131971 Person {'name': 'Jack'}
844424930131972 Person {'name': 'Andy', 'title': 'Developer'}
844424930131973 Person {'name': 'Tom', 'title': 'Developer'}
---[Parse all vertices in result set]---
Count of queried vertices: 1
---[Print vertices in Graph object]---
844424930131972 Person {'name': 'Andy', 'title': 'Developer'}


### Create & Query Edges

In [4]:
# Create Edges
age.execCypher(cursor, GRAPH_NAME, "MATCH (a:Person), (b:Person) WHERE a.name = 'Joe' AND b.name = 'Smith' CREATE (a)-[r:workWith {weight: 3}]->(b)")
age.execCypher(cursor, GRAPH_NAME, "MATCH (a:Person), (b:Person) WHERE  a.name = 'Andy' AND b.name = 'Tom' CREATE (a)-[r:workWith {weight: 1}]->(b)")
age.execCypher(cursor, GRAPH_NAME, "MATCH (a:Person {name: 'Jack'}), (b:Person {name: 'Andy'}) CREATE (a)-[r:workWith {weight: 5}]->(b)")
age.execCypher(cursor, GRAPH_NAME, "MATCH (a:Person {name: 'Joe'}), (b:Person {name: 'Jack'}) CREATE (a)-[r:workWith {weight: 5}]->(b)")
conn.commit()

print("-- [Query paths] --------")
age.queryCypher(cursor, GRAPH_NAME, "MATCH p=()-[:workWith]-() RETURN p")

print("---[Parse & print paths row by row]---")
for path in age.getRows(cursor):
    if len(path) == 3:
        print(path[0].label, path[0]["name"] , "-",  path[1].label, path[1]["weight"] , "-", path[2].label, path[2]["name"])

print("-- [Query & Print paths ] --------")
age.queryCypher(cursor, GRAPH_NAME, "MATCH p=()-[:workWith]-() RETURN p")

print("---[Parse & print paths row by row]---")
for path in age.getRows(cursor):
    print(path)


-- [Query paths] --------
---[Parse & print paths row by row]---
Person Smith - workWith 3 - Person Joe
Person Jack - workWith 5 - Person Joe
Person Joe - workWith 3 - Person Smith
Person Joe - workWith 5 - Person Jack
Person Andy - workWith 5 - Person Jack
Person Jack - workWith 5 - Person Andy
Person Tom - workWith 1 - Person Andy
Person Andy - workWith 1 - Person Tom
-- [Query & Print paths ] --------
---[Parse & print paths row by row]---
[{label:Person, id:844424930131970, properties:{name: Smith,}}::VERTEX,{label:workWith, id:1125899906842625, properties:{weight: 3,}, start_id:844424930131969, end_id:844424930131970}::EDGE,{label:Person, id:844424930131969, properties:{name: Joe,}}::VERTEX]::PATH
[{label:Person, id:844424930131971, properties:{name: Jack,}}::VERTEX,{label:workWith, id:1125899906842628, properties:{weight: 5,}, start_id:844424930131969, end_id:844424930131971}::EDGE,{label:Person, id:844424930131969, properties:{name: Joe,}}::VERTEX]::PATH
[{label:Person, id:84442

## Query Scalar or properties value

In [5]:
# Query scalar value
print("-- Query scalar value --------------------")
age.queryCypher(cursor, GRAPH_NAME, "MATCH (n:Person) RETURN id(n)")
value = age.getSingle(cursor)
print(value)
    
# Query properties 
print("-- Query properties --------------------")
age.queryCypher(cursor, GRAPH_NAME, "MATCH (n:Person) RETURN properties(n)")
value = age.getSingle(cursor)
print(value)
    
# Query properties value
print("-- Query properties value --------------------")
age.queryCypher(cursor, GRAPH_NAME, "MATCH (n:Person {name: 'Andy'}) RETURN n.title")
value = age.getSingle(cursor)
print(value)
    
print("-- Query path count --------")
age.queryCypher(cursor, GRAPH_NAME, "MATCH p=(:Person {name: 'Andy'})-[:workWith]-() RETURN count(p)")
graph = age.buildGraph(cursor)
print(graph[0])

-- Query scalar value --------------------
844424930131969
-- Query properties --------------------
{'name': 'Joe'}
-- Query properties value --------------------
Developer
-- Query path count --------
2


## Close connection

In [6]:
# Clear test data
cursor.execute("SELECT drop_graph(%s, true);", (GRAPH_NAME,))
conn.commit()

conn.close()